In [1]:
# imports
import polars as pl
import polars.selectors as cs
import os
from statistics_utils import ne2cc, build_country_code_dicts
import plotly.express as px
import json
import plotly.graph_objects as go
import pandas
import altair as alt
import geopandas as gpd
import contextily as cx
from shapely.geometry import Point, LineString, Polygon
pl.Config.set_tbl_rows(20)

polars.config.Config

In [2]:
def load_and_concat_parquet_files(path):
    dfs = []
    for file in os.listdir(path):
        file_path = os.path.join(path, file)
        if file_path.endswith('.parquet'):
            dfs.append(convert_types(pl.read_parquet(file_path).with_columns([pl.lit(file[0:2]).alias("ReferringCountry")])))
    if not dfs:
        return pl.DataFrame()
    combined_df = dfs[0]
    for df in dfs[1:]:
        combined_df = pl.concat([combined_df,df], how="diagonal")
    return combined_df

def convert_types(df):
    for col in df.columns:
        if df[col].dtype != pl.String:
            df = df.with_columns(df[col].cast(pl.String))
    return df

path = "../../data/filtered/"
df = load_and_concat_parquet_files(path)
df = df.drop("[0.002s][warning][perf,memops] Cannot use file /tmp/hsperfdata_kopp/3612625 because it is locked by another process (errno = 11)")

df = df.with_columns(
    pl.col("Date").str.strptime(pl.Date, "%Y-%m-%d")
)
print(df)

shape: (1_818_766, 27)
┌────────────┬───────────┬───────────┬─────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ text_id    ┆ name_type ┆ position  ┆ entity  ┆ … ┆ Speaker_n ┆ Speaker_g ┆ Speaker_b ┆ Referring │
│ ---        ┆ ---       ┆ ---       ┆ ---     ┆   ┆ ame       ┆ ender     ┆ irth      ┆ Country   │
│ str        ┆ str       ┆ str       ┆ str     ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│            ┆           ┆           ┆         ┆   ┆ str       ┆ str       ┆ str       ┆ str       │
╞════════════╪═══════════╪═══════════╪═════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ ParlaMint- ┆ LOC       ┆ ParlaMint ┆ europe  ┆ … ┆ Manuel    ┆ M         ┆ null      ┆ PT        │
│ PT_2015-03 ┆           ┆ -PT_2015- ┆         ┆   ┆ Lopes     ┆           ┆           ┆           │
│ -20.u83    ┆           ┆ 03-20.seg ┆         ┆   ┆ Moreira   ┆           ┆           ┆           │
│            ┆           ┆ 287…      ┆         ┆   ┆ da        ┆    

In [3]:
print(df.group_by("entity").len().sort("len", descending=True).filter())

shape: (270, 2)
┌────────────────────┬────────┐
│ entity             ┆ len    │
│ ---                ┆ ---    │
│ str                ┆ u32    │
╞════════════════════╪════════╡
│ europe             ┆ 337559 │
│ germany            ┆ 84267  │
│ russia             ┆ 81258  │
│ united states      ┆ 75534  │
│ ukraine            ┆ 60249  │
│ kosovo             ┆ 48416  │
│ france             ┆ 44566  │
│ spain              ┆ 40984  │
│ syria              ┆ 39111  │
│ china              ┆ 37667  │
│ …                  ┆ …      │
│ brunei darussalam  ┆ 3      │
│ east australia     ┆ 2      │
│ west america       ┆ 2      │
│ south west asia    ┆ 2      │
│ norfolk island     ┆ 1      │
│ türkiye            ┆ 1      │
│ northern australia ┆ 1      │
│ north east africa  ┆ 1      │
│ southern asia      ┆ 1      │
│ south west africa  ┆ 1      │
└────────────────────┴────────┘


In [4]:
dicts = build_country_code_dicts()
def ne2cc_s(x):
    return ne2cc(x, dicts[0], dicts[1])


In [5]:
df = df.with_columns([
    pl.col("entity").map_elements(ne2cc_s, return_dtype=pl.String).alias("ReferredCountry")
])

print(
df.filter(pl.col("ReferredCountry").is_null()).group_by("entity").agg(pl.count()).sort("count", descending=True)
)
df = df.filter(
    pl.col("ReferredCountry") != pl.col("ReferringCountry")
)

print(df)

/tmp/ipykernel_15704/3088195442.py:6: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  df.filter(pl.col("ReferredCountry").is_null()).group_by("entity").agg(pl.count()).sort("count", descending=True)


shape: (52, 2)
┌────────────────────┬────────┐
│ entity             ┆ count  │
│ ---                ┆ ---    │
│ str                ┆ u32    │
╞════════════════════╪════════╡
│ europe             ┆ 337559 │
│ eu                 ┆ 23911  │
│ america            ┆ 23429  │
│ africa             ┆ 18271  │
│ asia               ┆ 5775   │
│ north africa       ┆ 2814   │
│ south america      ┆ 1886   │
│ north america      ┆ 1231   │
│ west africa        ┆ 635    │
│ northern europe    ┆ 620    │
│ …                  ┆ …      │
│ northern africa    ┆ 4      │
│ brunei darussalam  ┆ 3      │
│ east australia     ┆ 2      │
│ south west asia    ┆ 2      │
│ west america       ┆ 2      │
│ north east africa  ┆ 1      │
│ south west africa  ┆ 1      │
│ türkiye            ┆ 1      │
│ southern asia      ┆ 1      │
│ northern australia ┆ 1      │
└────────────────────┴────────┘
shape: (1_375_796, 28)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬─────────

In [6]:
print(df.group_by("ReferredCountry").len().sort("len", descending=True).filter())

shape: (214, 2)
┌─────────────────┬───────┐
│ ReferredCountry ┆ len   │
│ ---             ┆ ---   │
│ str             ┆ u32   │
╞═════════════════╪═══════╡
│ US              ┆ 98246 │
│ DE              ┆ 84267 │
│ RU              ┆ 81258 │
│ UA              ┆ 60249 │
│ XK              ┆ 48416 │
│ FR              ┆ 44566 │
│ SY              ┆ 39111 │
│ CN              ┆ 37667 │
│ SE              ┆ 35132 │
│ GR              ┆ 34184 │
│ …               ┆ …     │
│ SX              ┆ 23    │
│ CZ              ┆ 21    │
│ SZ              ┆ 20    │
│ SB              ┆ 15    │
│ CD              ┆ 14    │
│ AS              ┆ 11    │
│ NU              ┆ 8     │
│ FK              ┆ 6     │
│ CX              ┆ 5     │
│ NF              ┆ 1     │
└─────────────────┴───────┘


In [7]:
alt.data_transformers.disable_max_rows()

df_yearly = df.group_by("ReferringCountry", "Date").len().sort("Date", descending=False)
df_yearly = df_yearly.with_columns(
    pl.col("Date").dt.truncate("1y").alias("year_start")
)

df_yearly = df_yearly.group_by("year_start", "ReferringCountry").agg([
    pl.col("len").sum().alias("year_sum")])
df_yearly.plot.line(x="year_start", y="year_sum").properties(width=100,height=100).facet(
    facet="ReferringCountry", columns=6
)

alt.FacetChart(...)

In [8]:
alt.data_transformers.disable_max_rows()

df_yearly = df.group_by("Date").len().sort("Date", descending=False)
df_yearly = df_yearly.with_columns(
    pl.col("Date").dt.truncate("1y").alias("year_start")
)

df_yearly = df_yearly.group_by("year_start").agg([
    pl.col("len").sum().alias("year_sum")])
df_yearly.plot.line(x="year_start", y="year_sum").properties(width=200,height=200)

alt.Chart(...)

In [9]:
df.group_by("ReferringCountry", "ReferredCountry").len().sort("len", descending=True)


ReferringCountry,ReferredCountry,len
str,str,u32
"""RS""","""XK""",40588
"""DK""","""GL""",20248
"""TR""","""SY""",12581
"""RS""","""ME""",11041
"""CZ""","""UA""",10945
"""NO""","""SE""",10889
"""NO""","""US""",10465
"""AT""","""DE""",10322
"""UA""","""RU""",8781


In [10]:
df.group_by("ReferringCountry", "ReferredCountry").len().sort("len", descending=True).filter(pl.col("ReferredCountry") == "UA")


ReferringCountry,ReferredCountry,len
str,str,u32
"""CZ""","""UA""",10945
"""NL""","""UA""",5656
"""SE""","""UA""",5303
"""PL""","""UA""",4990
"""EE""","""UA""",4819
"""DK""","""UA""",3492
"""HR""","""UA""",2712
"""BE""","""UA""",2589
"""HU""","""UA""",2504


In [11]:
total_counts = df.group_by("ReferringCountry").len().rename({"len": "total"})

# Step 2: Count pairs and compute proportion
result = (
    df.group_by("ReferringCountry", "ReferredCountry")
      .len()
      .join(total_counts, on="ReferringCountry")
      .with_columns(
          (pl.col("len") / pl.col("total")).alias("proportion")
      )
      .sort("proportion", descending=True)
)
result

ReferringCountry,ReferredCountry,len,total,proportion
str,str,u32,u32,f64
"""GB""","""LY""",1883,4218,0.44642
"""RS""","""XK""",40588,124159,0.326903
"""IS""","""US""",6892,26678,0.25834
"""UA""","""RU""",8781,38483,0.228179
"""DK""","""GL""",20248,94522,0.214215
"""BA""","""HR""",2611,12452,0.209685
"""FI""","""SE""",3468,21547,0.16095
"""GB""","""SS""",666,4218,0.157895
"""BA""","""RS""",1935,12452,0.155397


In [12]:
df.group_by("ReferringCountry", "ReferredCountry", "Party_orientation").len().sort("len", descending=True)


ReferringCountry,ReferredCountry,Party_orientation,len
str,str,str,u32
"""RS""","""XK""","""Far-right""",12603
"""RS""","""XK""","""Centre-left""",8144
"""DK""","""GL""","""Left""",5941
"""RS""","""XK""","""Big tent""",5764
"""RS""","""XK""","""Right""",5209
"""RS""","""XK""","""-""",4069
"""DK""","""GL""","""Centre-right""",3910
"""DK""","""GL""","""Centre-left""",3902
"""TR""","""SY""","""Centre-left""",3781


In [13]:
with open("restcountries_all.json", encoding="utf-8") as f:
    cntry_data = json.load(f)

def cca2ToGeo(ca):
    for country in cntry_data:
        if country["cca2"] == ca:
            return country["latlng"]
    return None

In [14]:
df_geo = result.with_columns([
    pl.col("ReferringCountry").map_elements(cca2ToGeo, return_dtype=pl.List(pl.Float64)).alias("ReferringCountryGeo"),
    pl.col("ReferredCountry").map_elements(cca2ToGeo, return_dtype=pl.List(pl.Float64)).alias("ReferredCountryGeo")
])


In [ ]:
import folium
df_geo_FI = df_geo.filter(pl.col("ReferringCountry") == "FI").to_pandas()
df_geo_FI["geometry"] =df_geo_FI.apply(
    lambda row: LineString([row["ReferringCountryGeo"].reverse(), row["ReferredCountryGeo"].reverse()]),
    axis=1
)
geo_df = gpd.GeoDataFrame(df_geo_FI, geometry='geometry', crs="EPSG:4326")
# Get center of your data for map start
geo_df.explore()